### Testing and Validation of a Network with Batfish

Network engineers often need to verify their networks via assertions on device configurations.  This can become a daunting task for large networks when done manually.  However, programmatically performing these assertions on the vendor-independent model produced by Batfish can be done quickly and easily.

In this notebook, we will look at different types of validations that can be done with Batfish.  More specifically, we will consider validation of _NTP servers_ and we will see how these validations are possible with [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/).

In [1]:
# Importing packages and loading questions
%run startup.py

### Initializing our Network and Snapshot

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"
SNAPSHOT_PATH = "../test_rigs/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME)

'{\n  "answerElements" : [\n    {\n      "class" : "org.batfish.datamodel.answers.InitInfoAnswerElement",\n      "parseStatus" : {\n        "as1border1" : "PASSED",\n        "as1border2" : "PASSED",\n        "as1core1" : "PASSED",\n        "as2border1" : "PASSED",\n        "as2border2" : "PASSED",\n        "as2core1" : "PASSED",\n        "as2core2" : "PASSED",\n        "as2dept1" : "PASSED",\n        "as2dist1" : "PASSED",\n        "as2dist2" : "PASSED",\n        "as3border1" : "PASSED",\n        "as3border2" : "PASSED",\n        "as3core1" : "PASSED",\n        "host1" : "PASSED",\n        "host2" : "PASSED",\n        "iptables/host1.iptables" : "PASSED",\n        "iptables/host2.iptables" : "PASSED"\n      }\n    }\n  ],\n  "status" : "SUCCESS",\n  "summary" : {\n    "numFailed" : 0,\n    "numPassed" : 0,\n    "numResults" : 0\n  }\n}\n'

The network snapshot that we initialized above would look like below. You can download/view devices' configuration files from [here](https://github.com/batfish/batfish/tree/master/test_rigs/example).

<img src="https://raw.githubusercontent.com/batfish/batfish/master/test_rigs/example/example-network.png" height=auto width=60%>

### Validating _NTP Servers_ Configuration
There are many potential assertions we might apply on _NTP server_ configuration, but in this notebook we will focus on the common scenarios we have observed in some real world networks.
In this exercise, we will try to validate the following scenarios with respect to a set of reference servers `[23.23.23.23]` (which contains just a single server to make things simple):
* Every node has some _NTP server_ configured (not necessarily from the reference servers).
* Every node has the same set of _NTP servers_ as the reference servers.
* Every node contains at least one _NTP server_ from the reference servers.

In [3]:
ref_ntp_servers = set(['23.23.23.23'])

We can restrict our validation to the border nodes to better simultate a real scenario. To start with, let's see the _NTP servers_ configured on all border nodes.

In [4]:
node_props = bfq.nodeProperties(nodeRegex=".*border.*", propertySpec="ntp-servers").answer().frame()
node_props

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


#### At least one _NTP server_ is configured on each node
This check verifies that every node has at least one _NTP server_, but does not check the address of the server.<br>To find violators, we can simply find nodes where the list of servers is empty, which is what the following command does:<br>(If you want to know more about the **lambda** keyword, see [lambda expressions](https://docs.python.org/3/reference/expressions.html#lambda).)

In [5]:
# Violators
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: len(x) == 0)]
ns_violators

,node,ntp-servers
0,as1border1,[]


#### _NTP servers_ on all nodes should be the same as our reference
A common use case for validating _NTP servers_ generally involves checking that the set of _NTP servers_ on all relevant nodes is equal to a given set. Doing this using **Pybatfish/pandas** is pretty straightforward.

An example command for getting a table of violators would look like

In [6]:
# Violators (Nodes whose set of NTP servers is not equal to our reference set)
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: ref_ntp_servers != set(x))]
ns_violators

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


As we can see, all border nodes violate this condition.

To look at which nodes actually have different _NTP servers_ compared to our reference set, we can do a set difference on the `ntp-servers` column. Commands to do that would look like below:

In [7]:
def abs_set_diff(a, b):
    return a - b if len(a - b) else b - a


ns_difference = node_props["ntp-servers"].map(lambda x: abs_set_diff(set(x), ref_ntp_servers))
# Let's pair it up with the node columns for a better view
diff_df = pd.DataFrame({'node': node_props["node"], 'ntp-servers-difference': ns_difference})
# Getting only the rows with a non-empty ntp-server-difference
diff_df[diff_df["ntp-servers-difference"].apply(lambda x: len(x) > 0)]

,node,ntp-servers-difference
0,as1border1,{23.23.23.23}
1,as1border2,{18.18.18.18}
2,as2border1,{18.18.18.18}
3,as2border2,{18.18.18.18}
4,as3border1,{18.18.18.18}
5,as3border2,{18.18.18.18}


#### _NTP servers_ on all nodes should contain at least one _NTP server_ from our reference
_NTP servers_ on all nodes should contain at least one _NTP server_ from our reference. This is a more lenient version of the previous check which verifies that every node has at least one _NTP server_ from our reference set.

To get a table of violators, we can use the following command.<br>If a node does not have even one common _NTP server_ with our reference set, we will count it as a violator.

In [8]:
# Violators (Nodes which do not contain even a single NTP server from our reference set)
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: len(ref_ntp_servers.intersection(set(x))) == 0)]
ns_violators

,node,ntp-servers
0,as1border1,[]
3,as2border2,[18.18.18.18]


So `as1border1` contains an empty set of NTP servers which clearly violates our assertion in this case.

We can perform similar validations on other node-properties as well, for example on _dns-servers_, _interfaces_, _ip-access-lists_ etc.

Hope this will help you get started doing validations using Pybatfish !